# Data Analysis of Hacker News Website

This notebook analyzes posts on Hacker News. The data can be found [here](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts). The data set contains one year of posts up to September 26, 2016.

## Goals
Analyze the comments below posts.

### Setup

In [3]:
# Make data import from path easy on any machine 

import os
from config.definitions import ROOT_DIR
# print(os.path.join(ROOT_DIR, "data", "HN_posts_year_to_Sep_26_2016.csv"))
path = os.path.join(ROOT_DIR, "data", "HN_posts_year_to_Sep_26_2016.csv")

Read in the data:

In [7]:
import csv
with open(path) as hacker_news:
    hacker_texts = csv.reader(hacker_news)
    hn = list(hacker_texts)


Inspection of first five rows:

In [8]:
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']]
